In [1]:
corpus = []
vocabulary = []    
topics = []
coherence = 0.0

class Topic:
    def __init__(self, index):
        self.index = index
        # self.sentences = []
        self.sentences = ""
        self.words = []
        self.coherence = 0.0



#STEP 1 
DATASET = 'clicr'
DATASET_PATH = '../datasets/'+DATASET+'/cases-titles.txt'
with open(DATASET_PATH) as f:
	corpus = f.readlines()


from sentence_transformers import SentenceTransformer

BERT_MODEL = "bert-base-cased"
# BERT_MODEL = "dmis-lab/biobert-base-cased-v1.1"
language_model = SentenceTransformer(BERT_MODEL)
vector_space = language_model.encode(corpus)

/opt/conda/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
No sentence-transformers model found with name bert-base-cased. Creating a new one with mean pooling.
/opt/conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
corpus = []
vocabulary = []    
topics = []
coherence = 0.0

class Topic:
    def __init__(self, index):
        self.index = index
        self.sentences = ""
        self.words = []
        self.coherence = 0.0



### STEP 1 
DATASET = 'clicr'
DATASET_PATH = '../datasets/'+DATASET+'/cases-titles.txt'
with open(DATASET_PATH) as f:
	corpus = f.readlines()


from sentence_transformers import SentenceTransformer

BERT_MODEL = "bert-base-cased"
language_model = SentenceTransformer(BERT_MODEL)
vector_space = language_model.encode(corpus)

# In order to use locally stored Language Models
# MODEL_PATH = '../../models/' + BERT_MODEL
# language_model = SentenceTransformer(MODEL_PATH)
# vector_space = language_model.encode(model.sentences)

### STEP 2
DISTANCE_THRESHOLD =300

from sklearn.cluster import AgglomerativeClustering

clustering_model = AgglomerativeClustering(linkage="ward", distance_threshold=DISTANCE_THRESHOLD, n_clusters=None)
clustering_model = clustering_model.fit(vector_space)
k = clustering_model.n_clusters_

topics = [Topic(i) for i in range(k)]

for i in range(len(corpus)):
    inferred_cluster_index = clustering_model.labels_[i]
    topics[inferred_cluster_index].sentences += corpus[i] + " "



### STEP 3
MAX_DF = 0.1

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

if (k > 1):
    c_MAX_DF = 0.5
    c_tfidf_model = TfidfVectorizer(max_df=c_MAX_DF, smooth_idf=True, use_idf=True)
    c_tfidf = c_tfidf_model.fit_transform([topic.sentences for topic in topics])
    c_tfidf_matrix = c_tfidf.toarray()
    words = c_tfidf_model.get_feature_names_out()
                    
    for i, topic in enumerate(topics):
        sorted_term_indexes = np.argsort(-1*c_tfidf_matrix[topic.index])
        topic.words = [words[j] for j in sorted_term_indexes]
else:
    c_tfidf_model = TfidfVectorizer(max_df=MAX_DF,smooth_idf=True, use_idf=True)
    c_tfidf = c_tfidf_model.fit_transform(corpus)
    c_tfidf_matrix = c_tfidf.toarray()
    words = c_tfidf_model.get_feature_names_out()
    
    mean_tfidf = np.array(c_tfidf_matrix.mean(axis=0)).flatten()
    sorted_term_indexes = np.argsort(-1*mean_tfidf)
    topics[0].words = [words[j] for j in sorted_term_indexes]


# VALIDATION
TOP_WORDS = 50

from gensim.models import CoherenceModel
from gensim.corpora import Dictionary

tfidf_model = TfidfVectorizer(max_df=MAX_DF, smooth_idf=True, use_idf=True)
tfidf = tfidf_model.fit_transform(corpus)
vocabulary = tfidf_model.get_feature_names_out()
terms_by_sentence = tfidf_model.inverse_transform(tfidf)
        
dictionary = Dictionary(terms_by_sentence)
cm = CoherenceModel(topics=[topic.words[:TOP_WORDS] for topic in topics], texts=terms_by_sentence, dictionary=dictionary, coherence='c_v', topn=TOP_WORDS)
coherence = cm.get_coherence()
coherence_per_topic = cm.get_coherence_per_topic()

for i, coherence in enumerate(coherence_per_topic):
    topics[i].coherence = coherence
    print("Topic ", str(i))
    print(topics[i].words[:TOP_WORDS])

print("Total coherence: ", cm.get_coherence())
print("Coherence by topic: ", cm.get_coherence_per_topic()) 

nro de words para k=1: 12126
top words:  50
Words considered by ctfidf:  12126
Total words:  12126


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Topic  0
['patient', 'syndrome', 'to', 'rare', 'case', 'as', 'unusual', 'acute', 'cause', 'disease', 'presenting', 'for', 'presentation', 'by', 'artery', 'after', 'following', 'induced', 'pulmonary', 'associated', 'treatment', 'cell', 'primary', 'carcinoma', 'tumour', 'diagnosis', 'old', 'bilateral', 'management', 'due', 'secondary', 'year', 'spontaneous', 'from', 'report', 'infection', 'renal', 'cancer', 'pain', 'severe', 'young', 'non', 'child', 'cyst', 'recurrent', 'coronary', 'abdominal', 'pregnancy', 'chronic', 'mimicking']
Total coherence:  0.1355571139830774
Coherence by topic:  [0.1355571139830774]


## Ordenar os tópicos por coerência

In [ ]:
from operator import itemgetter

# print(model.topics[0].coherence)
# print(model.topics[1].coherence)

topics = sorted(topics, key=lambda topic: topic.coherence, reverse=True)
# print(model.topics)
print(topics[1].coherence)
print(topics[1].sorted_terms[:10])
print(topics[1].sorted_tfidf[:10])

## Encontrar sentenças contendo determinadas palavras

In [ ]:
for t in model.topics:
    print('Topic ',t.cluster)

    print(t.sorted_terms[:10])
    print(t.sorted_tfidf[:10])
    
    print(t.sorted_terms[-1])
    print(t.sorted_tfidf[-1])

    print(len(t.sentences))
    
    # if any("osteogenesis" in s for s in t.sentences):
    #     print("aquiiiiiii")
    #     print(s)

    matches = [match for match in t.sentences if "arteriovenous" in match]
     
    print(matches)
    
    print(t.sorted_terms[-2])
    print(t.sorted_tfidf[-2])
    # print(len(t.sorted_terms))
    print('---------------')


---------------
# comparações feitas entre o ABT, LDA e Randômico

In [ ]:
0.4034653288931263


k = 1, 2,   200
TW 10,   tfidf=0.015   ctfidf=0.9
0.32, 0.51,     ,   0.36             

TW 10,   tfidf=0.01   ctfidf=0.5
meu = 0.32  ,  0.51,   0.35   
LDA = 0.38, 0.20, 0.39

TW 10, tfidf=0.95    ctfidf=0.5
meu 0.35, 0.51, 0.36
lda= 0.36, 0.35 ,0.37
random = 0.51, 0.52, 

TW 100, tfidf=0.015    ctfidf=0.5
meu 0.59  0.90 , 0.81
lda = 0.27, 0.36 0.80
random = 0.98, 0.93, 

# ----------
# em belem
TW 20, tfidf=0.015    ctfidf=0.99
meu  0.22, 0.65 , 0.41
lda = 0.31, 0.34,    , 0.62
random = 0.72  0.72      

TW 20, tfidf=0.015    ctfidf=0.5
meu 0.22, 0.64, 7=0.7096830441426468, 0.41
lda = 0.31, 0.34,    , 0.62
random = 0.72  0.72     

TW 20, tfidf=0.015    ctfidf=0.99
meu 0.22, 0.64, 7= 0.40, 0.48771604676442953
lda = 0.31, 0.34,    , 0.62
random = 0.72  0.72    

In [ ]:
import random

# TOP_WORDS = 10
k = 2

random_topics = []
for i in range(k):
    r = vocabulary.copy()
    # r = topics[i].words.copy()
    
    random.shuffle(r)
    random_topics.append(r[:TOP_WORDS])

print(random_topics)

cm = CoherenceModel(topics=random_topics, texts=terms_by_sentence, dictionary=dictionary, coherence="c_v",topn=TOP_WORDS)
coherence = cm.get_coherence()  # get coherence value
coherence_per_topic = cm.get_coherence_per_topic()

print(coherence)
print(coherence_per_topic)

In [ ]:
### test_topics = []

r1 = ['syndrome', 'rare', 'presenting', 'patient', 'carcinoma', 'case', 'cell', 'bilateral', 'spontaneous', 'unusual', 'cause', 'diagnosis', 'renal', 'primary', 'presentation', 'acute', 'management', 'pregnancy', 'cancer', 'following']
r2 = ['after', 'patient', 'following', 'vein', 'induced', 'injury', 'rare', 'thrombosis', 'treatment', 'pulmonary', 'secondary', 'primary', 'fracture', 'cause', 'syndrome', 'tumour', 'lymphoma', 'associated', 'haemorrhage', 'adult']
r3 = ['case', 'patient', 'report', 'unusual', 'review', 'is', 'literature', 'rare', 'syndrome', 'disease', 'cause', 'acute', 'not', 'two', 'induced', 'bilateral', 'right', 'coronary', 'it', 'pain']
r4 = ['old', 'year', 'artery', 'acute', 'patient', 'unusual', 'presentation', 'rare', 'presenting', 'syndrome', 'woman', 'cerebral', 'after', 'cell', 'disease', 'non', 'severe', 'cause', 'aneurysm', 'multiple']
r5 = ['patient', 'acute', 'treatment', 'following', 'disease', 'rare', 'infection', 'management', 'syndrome', 'associated', 'presenting', 'endocarditis', 'valve', 'after', 'obstruction', 'pulmonary', 'injury', 'intestinal', 'necrotising', 'severe']

test_topics.append(r1[:TOP_WORDS])
test_topics.append(r2[:TOP_WORDS])
test_topics.append(r3[:TOP_WORDS])
test_topics.append(r4[:TOP_WORDS])
test_topics.append(r5[:TOP_WORDS])
# test_topics.append(r6[:TOP_WORDS])
# test_topics.append(r7[:TOP_WORDS])

print(test_topics)

dictionary = Dictionary(terms_by_sentence)
cm = CoherenceModel(topics=test_topics, texts=terms_by_sentence, dictionary=dictionary, coherence="c_v",topn=TOP_WORDS)

coherence = cm.get_coherence()  # get coherence value
coherence_per_topic = cm.get_coherence_per_topic()

print(coherence)
print(coherence_per_topic)

---------------

### Álgebra sobre espaços de sentenças e de palavras

In [ ]:
val = np.inner(embeddings[0],embeddings[1])

# MODEL = 'biobert-base'
# MODEL = 'bert-base-cased'
WORD_MODEL = 'ACD'

WORD_EMBEDDINGS_TSV_INPUT_PATH = '../from_embeddings_to_tsv/output_from_1/' + WORD_MODEL + '/'

word_embeddings_dataframe = pd.read_csv(WORD_EMBEDDINGS_TSV_INPUT_PATH + 'word_embeddings.tsv', sep='\t',header=None)
word_embeddings_numpy = word_embeddings_dataframe.to_numpy()
word_embeddings = word_embeddings_numpy[:, :-1]

---------------------------------------------------------------------

#### Buscando as palavras (i.e., BERT terms) mais próximas de uma certa sentença

In [ ]:
WORD_EMBEDDINGS_TSV_INPUT_PATH = '../from_embeddings_to_tsv/output_from_1/' + WORD_MODEL + '/labels.tsv'

terms = []
with open(WORD_EMBEDDINGS_TSV_INPUT_PATH) as f:
    terms = f.readlines()

val = np.inner(embeddings[0],word_embeddings[1])
val

from numpy import argsort

dists = [''] * len(word_embeddings)
dist_indexes = [''] * len(dists)

for i, embedding in enumerate(word_embeddings):
    dist = np.inner(embeddings[1],embedding)
    dists[i] = dist
    
sorted_dists = sorted(dists)
args = argsort(dists)
        
with open(OUTPUT_PATH + '/mainterms.txt', "a") as file:
    print('Word: '+WORD_MODEL, file=file)  
    print('Sentence: ' + sentences[1], file=file)
    for a in args:
        if not terms[a].startswith('##'):
            print(terms[a], file=file)   
            i = i+1
            if i>100:
                break;

    print('----------------------------------------------------------------------------', file=file)  

sorted_dists[:10]
terms[17730]

sorted_topic_indexes = np.argsort(model.coherence_per_topic)
for i,j in enumerate(sorted_topic_indexes):    
    print(i, model.topics[j].coherence, model.topics[j].sorted_terms[:10])

---------------------------------------------

In [ ]:
### corrigir ordem dos termos. Ja esta corrigido no notebook oficial e no artigo
print('Topics in ascending sort of coherence.')
sorted_topic_indexes = np.argsort(coherence_per_topic)
for i,j in enumerate(sorted_topic_indexes):    
    print(i, topics[j].coherence, topics[j].sorted_terms[:10])